In [1]:
import yfinance as yf

In [ ]:
# get data from yahoo finance

In [ ]:
import yfinance as yf

In [2]:
msft = yf.Ticker("MSFT")
msft.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [5]:
history_data = msft.history(period="1d", interval="1m",
                                 start=None, end=None, prepost=False, actions=True,
                                 auto_adjust=True, back_adjust=False, repair=True, keepna=False,
                                 proxy=None, rounding=False, timeout=10,
                                 raise_errors=False)
print(type(history_data))

# Create a new DataFrame with specific columns
history_data_imp_columns = history_data.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']].copy()

# Rename columns of the new DataFrame
history_data_imp_columns_renamed = history_data_imp_columns.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'})



<class 'pandas.core.frame.DataFrame'>


,open,high,low,close,volume
Datetime,,,,,
2024-01-08 09:30:00-05:00,369.299988,370.060394,369.010010,369.390015,652292
2024-01-08 09:31:00-05:00,369.399994,369.635010,369.339996,369.459991,108600
2024-01-08 09:32:00-05:00,369.468903,369.744995,369.399994,369.622406,86102
2024-01-08 09:33:00-05:00,369.600098,369.989990,369.540009,369.899994,51614
2024-01-08 09:34:00-05:00,369.899994,369.989990,369.625000,369.760010,88358
...,...,...,...,...,...
2024-01-08 11:24:00-05:00,370.401489,370.519897,370.369995,370.429993,33827
2024-01-08 11:25:00-05:00,370.470001,370.540009,370.400085,370.400085,26049
2024-01-08 11:26:00-05:00,370.424988,370.500000,370.290009,370.299988,21443


In [ ]:
# Store Data in Influx DB

In [9]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime

url = "http://localhost:8086"
token = 'yx6JRL0nCzxw5Q67CQ2OPlFdJoVqo4cOHk7gRjfMY43cJlIzRXXNJ0MTB9K-EY7PwGRpq_uRKAAcDYsoagA5gg=='
org = 'trade_smart'
bucket = 'trade_smart'

def insert_dataframe(measurement=None, tag_dict=None, fields_dataframe=None, fields_list=[]):
    if measurement is None or tag_dict is None:
        return {"success": False, "error": "Incorrect input Arguments"}

    # # Convert DataFrame to InfluxDB line protocol format
    points = []
    if fields_dataframe is not None:
        for _, row in fields_dataframe.iterrows():
            point = Point(measurement)
            for tag_key, tag_value in tag_dict.items():
                point.tag(tag_key, tag_value)
            for column, value in row.items():
                point.field(column, value)
            point.time(_)
            points.append(point)
    if fields_list is not None or len(fields_list) > 0:
        for field in fields_list:
            point = Point(measurement)
            for tag_key, tag_value in tag_dict.items():
                point.tag(tag_key, tag_value)
            for key, val in field.items():
                point.field(key, val)
            points.append(point)

    # [p for p in points]
    line_protocol = "\n".join([p.to_line_protocol() for p in points])

    # Write financial_data to InfluxDB

    client = InfluxDBClient(url=url, token=token)
    write_api = client.write_api(write_options=SYNCHRONOUS)
    writer_resp = write_api.write(bucket=bucket, org=org, record=line_protocol)
    client.close()

    # Write financial_data to InfluxDB
    # with InfluxDBClient(url=self.url, token=self.token) as client:
    #     with client.write_api(write_options=SYNCHRONOUS) as write_api:
    #         write_api.write(bucket=self.bucket, org=self.org, record=line_protocol)

    return


for i in range(0, len(history_data_imp_columns_renamed), 100):
    chunk = history_data_imp_columns_renamed.iloc[i:i + 100]
    processed_chunk = insert_dataframe(measurement='candlestick', tag_dict={'symbol': "MSFT"}, fields_dataframe=chunk)
    print(processed_chunk)


None
None


In [ ]:
# Fetch data from Influx DB and draw indicators

In [21]:
import pandas as pd
url = "http://localhost:8086"
token = 'yx6JRL0nCzxw5Q67CQ2OPlFdJoVqo4cOHk7gRjfMY43cJlIzRXXNJ0MTB9K-EY7PwGRpq_uRKAAcDYsoagA5gg=='
org = 'trade_smart'
bucket = 'trade_smart'

def fetch_records(measurement):
    # Need to implement a query builder method
    client = InfluxDBClient(url=url, token=token)
    query = f'from(bucket: "{bucket}") |> range(start: 0) |> filter(fn: (r) => r._measurement == "{measurement}")'
    query = """
        from(bucket: "trade_smart")
          |> range(start: -24h)
          |> filter(fn: (r) => r["_measurement"] == "candlestick")
          |> filter(fn: (r) => r["symbol"] == "MSFT")
          |> yield(name: "mean")
    """
    print(query)
    tables = client.query_api().query(query, org=org)
    client.close()
    return tables

influx_resp = fetch_records("MSFT")
print(influx_resp)
data_frames = [pd.DataFrame(table.records) for table in influx_resp]
# Print the DataFrame(s)
display(data_frames)
# for i, df in enumerate(data_frames):
#     print(f"\nDataFrame {i + 1}:\n{df}")


        from(bucket: "trade_smart")
          |> range(start: -24h)
          |> filter(fn: (r) => r["_measurement"] == "candlestick")
          |> filter(fn: (r) => r["symbol"] == "MSFT")
          |> yield(name: "mean")
    
[<FluxTable: 9 columns, 119 records>, <FluxTable: 9 columns, 119 records>, <FluxTable: 9 columns, 119 records>, <FluxTable: 9 columns, 119 records>, <FluxTable: 9 columns, 119 records>]


[                                                     0
 0    FluxRecord() table: 0, {'result': 'mean', 'tab...
 1    FluxRecord() table: 0, {'result': 'mean', 'tab...
 2    FluxRecord() table: 0, {'result': 'mean', 'tab...
 3    FluxRecord() table: 0, {'result': 'mean', 'tab...
 4    FluxRecord() table: 0, {'result': 'mean', 'tab...
 ..                                                 ...
 114  FluxRecord() table: 0, {'result': 'mean', 'tab...
 115  FluxRecord() table: 0, {'result': 'mean', 'tab...
 116  FluxRecord() table: 0, {'result': 'mean', 'tab...
 117  FluxRecord() table: 0, {'result': 'mean', 'tab...
 118  FluxRecord() table: 0, {'result': 'mean', 'tab...
 
 [119 rows x 1 columns],
                                                      0
 0    FluxRecord() table: 1, {'result': 'mean', 'tab...
 1    FluxRecord() table: 1, {'result': 'mean', 'tab...
 2    FluxRecord() table: 1, {'result': 'mean', 'tab...
 3    FluxRecord() table: 1, {'result': 'mean', 'tab...
 4    FluxRecord() ta

In [ ]:
# Store Indicator in Influx DB


In [ ]:
# Trade